# Data Dictionary
#### DATE_TIME 
It is the field that contains date and time information. Data format YYYY-MM-DD HH24: MI: SS format. The date break is hourly.
#### OBSERVATORY_NAME
It indicates the type number of the sensor where the measurement is made.
#### AVERAGE_TEMPERATURE
Average temperature (& ordm; C) measured from the respective sensor at the given hour.
#### AVERAGE_HUMIDITY
Average wind speed (km / h) measured from the relevant sensor for the given hour.
#### AVERAGE_WIND
Average wind direction (km / h) measured from the respective sensor at the given hour.
#### AVERAGE_DIRECTIONOFWIND
Average precipitation amount (kg / m²) measured from the relevant sensor in the given hour.
#### AVERAGE_PRECIPITATION
Average amount of precipitation (kg / m²) measured from the relevant sensor in the given hour.
#### AVERAGE_ROAD_TEMPERATURE
Average road temperature (& ordm; C) measured from the respective sensor at the given hour.

In [18]:
import os
import pandas as pd
import numpy as np

import locale
from locale import atof
import xlsxwriter

from PreProcessingUtil import preprocessing

In [19]:
# Select rows in a DataFrame between two dates
def selectedDateFrame(dfPol, dfMet):
    dfPol = dfPol[(startDate<=dfPol['Tarih']) & (dfPol['Tarih']<endDate)]
    dfMet = dfMet[(startDate<=dfMet['DATE_TIME']) & (dfMet['DATE_TIME']<endDate)]
    return dfPol, dfMet

# Function to fill missing rows
def fillingEmptyRows(dfTrue,dfFalse):
    for x in range (dfTrue.shape[0]):
        if not (str(dfTrue['Tarih'].iloc[x])[:13] == str(dfFalse['DATE_TIME'].iloc[x])[:13]):
            temp = dfFalse.iloc[x-1] # get previous value
            temp['DATE_TIME'] = dfTrue['Tarih'].iloc[x] # set true date
            dfFalse = Insert_row(x, dfFalse, temp) # insert missing value        
    return dfTrue, dfFalse

# Function to insert row in the dataframe
def Insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
   
    # End value of upper half
    end_upper = row_number
   
    # Start value of lower half
    start_lower = row_number
   
    # End value of lower half
    end_lower = df.shape[0]
   
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
   
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
   
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
   
    # Combine the two lists
    index_ = upper_half + lower_half
   
    # Update the index of the dataframe
    df.index = index_
   
    # Insert a row at the end
    df.loc[row_number] = row_value
    
    # Sort the index labels
    df = df.sort_index()
   
    # return the dataframe
    return df

In [20]:
# creating dataframes per town
df1  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202001.csv"))
df2  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202002.csv"))
df3  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202003.csv"))
df4  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202004.csv"))
df5  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202005.csv"))
df6  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202006.csv"))
df7  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202007.csv"))
df8  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202008.csv"))
df9  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202009.csv"))
df10 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202010.csv"))
df11 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202011.csv"))
df12 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202012.csv"))
df13 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202101.csv"))
df14 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202102.csv"))

In [21]:
# concat all dataframes
bigdata = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14], ignore_index=True, sort=False)

In [22]:
# distinct town names
bigdata['OBSERVATORY_NAME'].unique()

array(['BEYKOZ_ANADOLU_FENERI', 'ISTOC_MAHMUTBEY',
       'GUNGOREN_DAVUTPASA_MARMARA', 'KILYOS', 'SILE_DARLIK', 'PASAKOY',
       'SANCAKTEPE_MGM', 'ESENLER', 'HACIOSMAN_SARIYER', 'SILE_ISAKOY',
       'DURUSU', 'SUBASI', 'CAMLICA_TUNEL_K', 'BAYRAMOGLU_TUZLA',
       'KAMILOBA', 'CAMLICA_LIBADIYE', 'SARIYER_ITU_MASLAK', 'CATALCA',
       'BUYUKADA', 'SABIHAGOKCEN', 'EMINONU', 'OLIMPIYAT', 'G_O_PASA',
       'BESIKTAS_YILDIZ', 'SILIVRI_ORMAN_SAHASI', 'USKUDAR_MGM', 'SILE',
       'RIVA_TUNEL_G', 'BUYUKCEKMECE_MGM', 'KARTAL_AYDOS_DAGI',
       'UMRANIYE', 'ODAYERI', 'ARNAVUTKOY_MGM', 'HADIMKOY', 'BEYKOZ',
       'SUREYYAPASA', 'SAMANDIRA', 'SARIYER', 'AKOM', 'CATALCA_MGM',
       'B_CEKMECE_SVIRAJLARI', 'BAHCESEHIR_I_KULE', 'SILIVRI_MGM',
       'TERKOS', 'CAVUSBASI', 'CANTA', 'TERKOS_BARAJI', 'PENDIK',
       'CEKMEKOY_OMERLI_MGM', 'SISLİ_MGM', 'KARTAL', 'CIFTALAN', 'SILE_2',
       'KADIKOY_GOZTEPE_MGM', 'GOZTEPE', 'BEYLİKDUZU_MGM', 'YSS_KOPRUSU',
       'BEYKOZ_MGM', 'AHL_BAKIRKOY', 

In [23]:
# selected columns
reducedData = bigdata[["DATE_TIME", "OBSERVATORY_NAME", "AVERAGE_TEMPERATURE", "AVERAGE_HUMIDITY", "AVERAGE_WIND", "AVERAGE_DIRECTIONOFWIND", "AVERAGE_PRECIPITATION", "AVERAGE_ROAD_TEMPERATURE"]]

In [24]:
# town names different in meteorology and pollutant datasets, so find and give the convenient
polTownName = 'umraniye'
metTownName = 'UMRANIYE'

In [25]:
# all pollutant values according to a particular town (sorted by date time)
polPath = polTownName + '.xlsx'
dfPollutant = pd.read_excel(os.path.join("../", "datasets", "pollutants", polPath),engine='openpyxl',parse_dates=True,thousands='.')

In [26]:
# all meteorology values according to a particular town (sorted by date time)
dfMeteorology = reducedData[reducedData['OBSERVATORY_NAME']==metTownName].sort_values('DATE_TIME')

In [27]:
# determining dates
startDate = '2020-01-01 00:00:00'
endDate = '2021-02-21 00:00:00'

In [28]:
# pooling selected rows according to determined date time
dfPol, dfMet = selectedDateFrame(dfPollutant, dfMeteorology)

In [29]:
# filling the missing rows with previous row value
dfPol, dfMet = fillingEmptyRows(dfPol, dfMet)

ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
# making preprocessing (fixing values, filling empty rows)
dfPol = preprocessing(dfPol)

In [31]:
dfPol

,Tarih,PM10 ( µg/m3 ),SO2 ( µg/m3 ),CO ( µg/m3 ),NO2 ( µg/m3 ),NOX ( µg/m3 ),O3 ( µg/m3 ),PM 2.5 ( µg/m3 )
70127,2020-01-01 00:00:56,44.400002,6.300000,0,92.199997,217.399994,3.100000,33.400002
70128,2020-01-01 01:00:56,41.599998,5.900000,0,95.300003,223.600006,3.000000,39.599998
70129,2020-01-01 02:00:56,59.500000,5.900000,0,78.000000,141.500000,6.800000,42.200001
70130,2020-01-01 03:00:56,45.500000,5.500000,0,51.700001,12.800000,14.600000,36.599998
70131,2020-01-01 04:00:56,25.700001,5.900000,0,82.699997,90.599998,5.600000,34.000000
...,...,...,...,...,...,...,...,...
80130,2021-02-20 19:00:56,53.299999,52.599998,0,158.699997,26.500000,32.200001,9.300000
80131,2021-02-20 20:00:56,45.700001,49.700001,0,180.000000,32.000000,47.299999,9.700000
80132,2021-02-20 21:00:56,44.900002,7.300000,0,178.300003,31.600000,16.600000,12.100000
80133,2021-02-20 22:00:56,33.099998,7.500000,0,147.300003,28.000000,28.500000,9.000000


In [32]:
# reset index values
dfMet = dfMet.reset_index(drop=True)
dfPol = dfPol.reset_index(drop=True)

In [33]:
# concat dataframes
result = pd.concat([dfPol, dfMet], axis=1, join='inner')
display(result)

,Tarih,PM10 ( µg/m3 ),SO2 ( µg/m3 ),CO ( µg/m3 ),NO2 ( µg/m3 ),NOX ( µg/m3 ),O3 ( µg/m3 ),PM 2.5 ( µg/m3 ),DATE_TIME,OBSERVATORY_NAME,AVERAGE_TEMPERATURE,AVERAGE_HUMIDITY,AVERAGE_WIND,AVERAGE_DIRECTIONOFWIND,AVERAGE_PRECIPITATION,AVERAGE_ROAD_TEMPERATURE
0,2020-01-01 00:00:56,44.400002,6.300000,0,92.199997,217.399994,3.100000,33.400002,2020-01-01 00:00:00,UMRANIYE,7.56148,99.99792,0.00000,258.60417,0.0,5.13750
1,2020-01-01 01:00:56,41.599998,5.900000,0,95.300003,223.600006,3.000000,39.599998,2020-01-01 01:00:00,UMRANIYE,7.42827,99.99831,0.00000,267.05085,0.0,4.90000
2,2020-01-01 02:00:56,59.500000,5.900000,0,78.000000,141.500000,6.800000,42.200001,2020-01-01 02:00:00,UMRANIYE,6.87673,99.76102,0.00000,270.44068,0.0,4.62203
3,2020-01-01 03:00:56,45.500000,5.500000,0,51.700001,12.800000,14.600000,36.599998,2020-01-01 03:00:00,UMRANIYE,6.01190,96.33966,0.00000,282.00000,0.0,4.27586
4,2020-01-01 04:00:56,25.700001,5.900000,0,82.699997,90.599998,5.600000,34.000000,2020-01-01 04:00:00,UMRANIYE,5.97140,97.14167,0.00000,282.00000,0.0,4.66500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10003,2021-02-20 19:00:56,53.299999,52.599998,0,158.699997,26.500000,32.200001,9.300000,2021-02-20 19:00:00,UMRANIYE,3.03206,100.00000,2.09898,63.18367,0.0,1.15714
10004,2021-02-20 20:00:56,45.700001,49.700001,0,180.000000,32.000000,47.299999,9.700000,2021-02-20 20:00:00,UMRANIYE,3.08695,100.00000,2.49364,70.89091,0.0,0.76000
10005,2021-02-20 21:00:56,44.900002,7.300000,0,178.300003,31.600000,16.600000,12.100000,2021-02-20 21:00:00,UMRANIYE,3.08978,100.00000,2.23963,60.50000,0.0,0.54630
10006,2021-02-20 22:00:56,33.099998,7.500000,0,147.300003,28.000000,28.500000,9.000000,2021-02-20 22:00:00,UMRANIYE,2.67189,100.00000,1.54585,46.00000,0.0,0.13774


In [34]:
# create excel writer object
path = '../datasets/training/'+polTownName+'_combined.xlsx'
writer = pd.ExcelWriter(path)
# write dataframe to excel
result.to_excel(writer)
# save the excel
writer.save()